In [ ]:
!pip install d2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached d2l-0.17.6-py3-none-any.whl (112 kB)
  Using cached jupyter-1.0.0-py2.py3-none-any.whl (2.7 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached pandas-1.2.4.tar.gz (5.5 MB)
  Installing build dependencies ... canceled
ERROR: Operation cancelled by user
^C


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install optuna
%cd /content/drive/MyDrive/EE-519/MP1

import numpy as np
import math
import torch
import optuna
import torchvision
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader, Dataset
import torch as d2l
from optuna.trial import TrialState

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.3 MB/s eta 0:00:00
/content/drive/MyDrive/EE-519/MP1


In [ ]:
!pip install scipy
import scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 2, 7)
    layers = []

    in_features = 16
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 10, 500)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    #layers.append(nn.LogSoftmax(dim=1))
    def init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform(m.weight)
            m.bias.data.fill_(0.01)

    net = nn.Sequential(*layers)  
    net.apply(init_weights)
    return net 

In [ ]:
def objective(trial):
    # Generate the model.
    model = define_model(trial)
    model = model.type(torch.float64)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    criterion = nn.HuberLoss()

    # Get the dataset.
    #train_loader, valid_loader, mu, nComp, pca_components, saved_labels, num_epochs = get_data()

    # Training of the model.
    for epoch in range(num_epochs):

        for i, sample in enumerate(train_loader):
            inputs, labels = sample[:,:-1], sample[:,-1]
            inputs = inputs.type(torch.float64)
            labels = labels.type(torch.float64)
            #zero gradients here instead??
            #forward pass
            outputs = model(inputs)
            #loss
            labels = labels.type(torch.float64)
            outputs = outputs.type(torch.float64)
            loss = criterion(outputs, labels)
            #zero gradients
            optimizer.zero_grad()
            #backward pass
            loss.mean().backward()
            optimizer.step()


        # Validation of the model.
        model.eval()
        correct = 0
        total = 0

        #tensor to store outputs
        FINAL_OUTPUTS = torch.zeros(0, dtype=torch.float32)

        with torch.no_grad():
            #for sample in valid_loader:
            inputs = valid_data[:,:-1]#, valid_data[:,-1]
            FINAL_OUTPUTS = torch.cat((FINAL_OUTPUTS, model(inputs)), 0)
            #labels = labels.reshape(FINAL_OUTPUTS.size())

        #set val_data = to valid_data
        outputs_data = valid_data
        #reshape outputs to put into val_data
        outputs_data[:, -1] = torch.zeros([611,], dtype=torch.float32)
        outputs_data[:, -1] = FINAL_OUTPUTS.reshape([611,])
        outputs_data = np.dot(pca.transform(outputs_data)[:, :nComp], pca_components[:nComp, :])
        outputs = outputs_data[:,-1]

        labels = saved_labels.to_numpy()

        def destandardize (Y, mean, std):
            input_destdize = (Y * std) + mean
            return input_destdize

        labels = destandardize(labels, denorm_mean, denorm_std)
        outputs = destandardize(outputs, denorm_mean, denorm_std)

        """
        OP_data = valid_data.numpy()
        OP_data[:,-1] = outputs
        OP_data = trans_valid.inverse_transform(np.atleast_2d(OP_data))
        outputs = OP_data[:,-1]

        OP_data = valid_data.numpy()
        OP_data[:,-1] = labels
        OP_data = trans_valid.inverse_transform(np.atleast_2d(OP_data))
        lables = OP_data[:,-1]
        """

        for i in range(len(outputs)):
            if outputs[i] <= 25:
                outputs[i] = 0
            elif outputs[i] <= 50 and outputs[i] > 25:
                outputs[i] = 1
            elif outputs[i] <= 200 and outputs[i] > 50:
                outputs[i] = 2
            else:
                outputs[i] = 3
            if labels[i] <= 25:
                labels[i] = 0
            elif labels[i] <= 50 and labels[i] > 25:
                labels[i] = 1
            elif labels[i] <= 200 and labels[i] > 50:
                labels[i] = 2
            else:
                labels[i] = 3

        #print(outputs)
        finals = pd.DataFrame()
        finals['outputs'] = outputs
        finals['labels'] = labels
        
        total = len(outputs)
        correct = (outputs == labels).sum().item()
        accuracy = correct / total
        
        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        return accuracy


def objective_outputs(trial):
    # Generate the model.
    model = define_model(trial)
    model = model.type(torch.float64)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    criterion = nn.HuberLoss()

    # Get the dataset.
    #train_loader, valid_loader, mu, nComp, pca_components, saved_labels, num_epochs = get_data()

    # Training of the model.
    for epoch in range(num_epochs):

        for i, sample in enumerate(train_loader):
            inputs, labels = sample[:,:-1], sample[:,-1]
            inputs = inputs.type(torch.float64)
            labels = labels.type(torch.float64)
            #zero gradients here instead??
            #forward pass
            outputs = model(inputs)
            #loss
            labels = labels.type(torch.float64)
            outputs = outputs.type(torch.float64)
            loss = criterion(outputs, labels)
            #zero gradients
            optimizer.zero_grad()
            #backward pass
            loss.mean().backward()
            optimizer.step()


        # Validation of the model.
        model.eval()
        correct = 0
        total = 0

        #tensor to store outputs
        FINAL_OUTPUTS = torch.zeros(0, dtype=torch.float32)

        with torch.no_grad():
            #for sample in valid_loader:
            inputs = valid_data[:,:-1]#, valid_data[:,-1]
            FINAL_OUTPUTS = torch.cat((FINAL_OUTPUTS, model(inputs)), 0)
            #labels = labels.reshape(FINAL_OUTPUTS.size())

        #set val_data = to valid_data
        outputs_data = valid_data
        #reshape outputs to put into val_data
        outputs_data[:, -1] = torch.zeros([611,], dtype=torch.float32)
        outputs_data[:, -1] = FINAL_OUTPUTS.reshape([611,])
        outputs_data = np.dot(pca.transform(outputs_data)[:, :nComp], pca_components[:nComp, :])
        outputs = outputs_data[:,-1]

        labels = saved_labels.to_numpy()

        def destandardize (Y, mean, std):
            input_destdize = (Y * std) + mean
            return input_destdize

        labels = destandardize(labels, denorm_mean, denorm_std)
        outputs = destandardize(outputs, denorm_mean, denorm_std)
        
        """
        OP_data = valid_data.numpy()
        OP_data[:,-1] = outputs
        OP_data = trans_valid.inverse_transform(np.atleast_2d(OP_data))
        outputs = OP_data[:,-1]

        OP_data = valid_data.numpy()
        OP_data[:,-1] = labels
        OP_data = trans_valid.inverse_transform(np.atleast_2d(OP_data))
        lables = OP_data[:,-1]
        """

        for i in range(len(outputs)):
            if outputs[i] <= 25:
                outputs[i] = 0
            elif outputs[i] <= 50 and outputs[i] > 25:
                outputs[i] = 1
            elif outputs[i] <= 200 and outputs[i] > 50:
                outputs[i] = 2
            else:
                outputs[i] = 3
            if labels[i] <= 25:
                labels[i] = 0
            elif labels[i] <= 50 and labels[i] > 25:
                labels[i] = 1
            elif labels[i] <= 200 and labels[i] > 50:
                labels[i] = 2
            else:
                labels[i] = 3
        
        finals = pd.DataFrame()
        finals['outputs'] = outputs
        finals['labels'] = labels
        
        total = len(outputs)
        correct = (outputs == labels).sum().item()
        accuracy = correct / total

        RMSE = np.sqrt(np.mean((labels - outputs)**2))
        
        l1norm = np.mean(np.absolute(outputs-labels))

        return finals, accuracy, RMSE, l1norm







In [ ]:
DEVICE = torch.device('cpu')
num_epochs = 10000
batch_size = 350
num_workers = 0
CLASSES = 1


dataset = pd.read_csv('data/heatflow_resid_test.csv')
#dataset.loc[dataset['hfqc_resid'] > 500, 'hfqc_resid'] = 500

from scipy import stats
cols_to_modify = [col for col in dataset.columns if col != 'hfqc_resid']

# Define a function to remove outliers based on the IQR method
def remove_outliers(data, columns):
    for col in columns:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        upper_threshold = q3 + 1.5 * iqr
        lower_threshold = q1 - 1.5 * iqr
        data[col] = data[col].apply(lambda x: x if lower_threshold <= x <= upper_threshold else data[col].median())
    return data

# Remove outliers from the dataset (excluding hfqc_resid)
modified_dataset = remove_outliers(dataset[cols_to_modify], cols_to_modify)

# Add hfqc_resid back to the modified dataset

#modified_dataset['hfqc_resid'] = dataset['hfqc_resid']

dataset = modified_dataset



train_data = dataset[['hf22_tgwt','cond_sur','cond_lcr','cond_man','cond_mcr','cond_uma','geop_mag','geop_grv','geop_dtb','eqi200n5','eqd200n5','geod_shr', 'geod_dil','geod_2nd','faultAl','vent_ing']], 'hfqc_resid']]
X_train = train_data.drop(['hfqc_resid'], axis=1)

#convert into test and validation sets
valid_data = train_data.iloc[2500:-1, :]
train_data = train_data.iloc[0:2500, :]

"""
cols = train_data.columns.to_series()
trans_training = QuantileTransformer(n_quantiles=100, output_distribution='normal')
train_data = trans_training.fit_transform(train_data)
train_data = pd.DataFrame(train_data, columns=cols)

cols = valid_data.columns.to_series()
trans_valid = QuantileTransformer(n_quantiles=100, output_distribution='normal')
valid_data = trans_valid.fit_transform(valid_data)
valid_data = pd.DataFrame(valid_data, columns=cols)
"""

#normalize data
mean = train_data.mean()
std = train_data.std() 
denorm_mean = mean['hfqc_resid']
denorm_std = std['hfqc_resid']

train_data = (train_data - mean) / std
np.reshape(train_data, [2500,17])
valid_data = (valid_data - mean) / std
np.reshape(valid_data, [611,17])

saved_labels = valid_data['hfqc_resid']

#Apply PCA
components = 10#our number of features we want to reduce to
pca = PCA(n_components=components)
pca.fit(train_data)
train_features = pd.DataFrame(pca.components_)
#use a for loop to replace all the values in the dataframe with the new values
for i in range(components):
     train_data.iloc[i,:] = train_features.iloc[i,:]


#Apply PCA to validation set
components = 10#our number of features we want to reduce to
pca = PCA(n_components=components)
pca.fit(valid_data)
stored_pca = valid_data
pca_components = pca.components_
valid_features = pd.DataFrame(pca.components_)
#use a for loop to replace all the values in the dataframe with the new values
for i in range(components):
     valid_data.iloc[i,:] = valid_features.iloc[i,:]

mu = np.mean(valid_data)
mu = mu['hfqc_resid']

nComp = 10

#convert datasets to tensors
tensored_data = torch.tensor(train_data.values, dtype=torch.float32)
valid_data = torch.tensor(valid_data.values, dtype=torch.float32)


valid_data = valid_data.type(torch.float64)
tensored_data = tensored_data.type(torch.float64)
#load data into dataloader
train_loader = torch.utils.data.DataLoader(dataset=tensored_data, batch_size=batch_size, shuffle=True)#, num_workers=num_workers
valid_loader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=True)#, num_workers=num_workers


SyntaxError: ignored

In [ ]:
TESTER = 0
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, timeout=600)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-05-02 03:36:16,151] A new study created in memory with name: no-name-7dd883a5-af50-48b0-b5cb-561b2fdbf9b8
<ipython-input-2-4099f84cef2c>:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:988: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:988: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta

Study statistics: 
  Number of finished trials:  200
  Number of pruned trials:  160
  Number of complete trials:  40
Best trial:
  Value:  0.37315875613747956
  Params: 
    n_layers: 4
    n_units_l0: 72
    dropout_l0: 0.4926871919304731
    n_units_l1: 424
    dropout_l1: 0.4999975227529518
    n_units_l2: 393
    dropout_l2: 0.27857632028981494
    n_units_l3: 301
    dropout_l3: 0.27446146011966965
    optimizer: Adam
    lr: 0.0001105720679317094


In [ ]:

def objective_outputs_test(trial):
    # Generate the model.
    model = define_model(trial)
    model = model.type(torch.float64)

    #tensor to store outputs
    FINAL_OUTPUTS = torch.zeros(0, dtype=torch.float32)

    with torch.no_grad():
        #for sample in valid_loader:
        inputs = tensored_data[:,:-1]#, valid_data[:,-1]
        FINAL_OUTPUTS = torch.cat((FINAL_OUTPUTS, model(inputs)), 0)
        #labels = labels.reshape(FINAL_OUTPUTS.size())

    #set val_data = to valid_data
    outputs_data = tensored_data
    #reshape outputs to put into val_data
    outputs_data[:, -1] = torch.zeros([778,], dtype=torch.float32)
    outputs_data[:, -1] = FINAL_OUTPUTS.reshape([778,])
    outputs_data = np.dot(pca.transform(outputs_data)[:, :nComp], pca_components[:nComp, :])
    outputs = outputs_data[:,-1]

    #labels = saved_labels.to_numpy()

    #def destandardize (Y, mean, std):
    #    input_destdize = (Y * std) + mean
    #    return input_destdize

    #labels = destandardize(labels, denorm_mean, denorm_std)
    #outputs = destandardize(outputs, denorm_mean, denorm_std)
    
    """
    OP_data = valid_data.numpy()
    OP_data[:,-1] = outputs
    OP_data = trans_valid.inverse_transform(np.atleast_2d(OP_data))
    outputs = OP_data[:,-1]

    OP_data = valid_data.numpy()
    OP_data[:,-1] = labels
    OP_data = trans_valid.inverse_transform(np.atleast_2d(OP_data))
    lables = OP_data[:,-1]
    """

    for i in range(len(outputs)):
        if outputs[i] <= 25:
            outputs[i] = 0
        elif outputs[i] <= 50 and outputs[i] > 25:
            outputs[i] = 1
        elif outputs[i] <= 200 and outputs[i] > 50:
            outputs[i] = 2
        else:
            outputs[i] = 3
    
    finals = pd.DataFrame()
    finals['outputs'] = outputs

    return finals

In [ ]:
DEVICE = torch.device('cpu')
num_epochs = 10000
batch_size = 350
num_workers = 0
CLASSES = 1


dataset = pd.read_csv('data/heatflow_resid_test.csv')
#dataset.loc[dataset['hfqc_resid'] > 500, 'hfqc_resid'] = 500

from scipy import stats
cols_to_modify = [col for col in dataset.columns if col != 'hfqc_resid']

# Define a function to remove outliers based on the IQR method
def remove_outliers(data, columns):
    for col in columns:
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        upper_threshold = q3 + 1.5 * iqr
        lower_threshold = q1 - 1.5 * iqr
        data[col] = data[col].apply(lambda x: x if lower_threshold <= x <= upper_threshold else data[col].median())
    return data

# Remove outliers from the dataset (excluding hfqc_resid)
modified_dataset = remove_outliers(dataset[cols_to_modify], cols_to_modify)

# Add hfqc_resid back to the modified dataset

#modified_dataset['hfqc_resid'] = dataset['hfqc_resid']

dataset = modified_dataset



train_data = dataset[['hf22_tgwt','cond_sur','cond_lcr','cond_man','cond_mcr','cond_uma','geop_mag','geop_grv','geop_dtb','eqi200n5','eqd200n5','geod_shr', 'geod_dil','geod_2nd','faultAl','vent_ing']]#, 'hfqc_resid']]
#X_train = train_data.drop(['hfqc_resid'], axis=1)

#convert into test and validation sets
#valid_data = train_data.iloc[2500:-1, :]
#train_data = train_data.iloc[0:2500, :]

"""
cols = train_data.columns.to_series()
trans_training = QuantileTransformer(n_quantiles=100, output_distribution='normal')
train_data = trans_training.fit_transform(train_data)
train_data = pd.DataFrame(train_data, columns=cols)

cols = valid_data.columns.to_series()
trans_valid = QuantileTransformer(n_quantiles=100, output_distribution='normal')
valid_data = trans_valid.fit_transform(valid_data)
valid_data = pd.DataFrame(valid_data, columns=cols)
"""

#normalize data
#mean = train_data.mean()
#std = train_data.std() 
#denorm_mean = mean
denorm_std = std

#train_data = (train_data - mean) / std
#np.reshape(train_data, [778,16])


#valid_data = (valid_data - mean) / std
#np.reshape(valid_data, [611,17])

#saved_labels = valid_data['hfqc_resid']
train_data['hfqc_resid'] = 1

#Apply PCA
components = 10#our number of features we want to reduce to
pca = PCA(n_components=components)
pca.fit(train_data)
train_features = pd.DataFrame(pca.components_)
#use a for loop to replace all the values in the dataframe with the new values
for i in range(components):
     train_data.iloc[i,:] = train_features.iloc[i,:]

"""
#Apply PCA to validation set
components = 10#our number of features we want to reduce to
pca = PCA(n_components=components)
pca.fit(valid_data)
stored_pca = valid_data
pca_components = pca.components_
valid_features = pd.DataFrame(pca.components_)
#use a for loop to replace all the values in the dataframe with the new values
for i in range(components):
     valid_data.iloc[i,:] = valid_features.iloc[i,:]

mu = np.mean(train_data)
mu = mu['hfqc_resid']
"""
nComp = 10


#convert datasets to tensors

tensored_data = torch.tensor(train_data.values, dtype=torch.float32)
#valid_data = torch.tensor(valid_data.values, dtype=torch.float32)


#valid_data = valid_data.type(torch.float64)

tensored_data = tensored_data.type(torch.float64)
#load data into dataloader

train_loader = torch.utils.data.DataLoader(dataset=tensored_data, batch_size=batch_size, shuffle=True)#, num_workers=num_workers
#valid_loader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=batch_size, shuffle=True)#, num_workers=num_workers


<ipython-input-29-b6d38269d2b8>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['hfqc_resid'] = 1


In [ ]:
awesome = objective_outputs_test(study.best_trial)
test_labels = awesome['outputs']
new_dataset = pd.read_csv('data/heatflow_resid_test.csv')
new_dataset['hfqc_resid'] = awesome['outputs']
new_dataset.to_csv('test.csv')  

<ipython-input-2-4099f84cef2c>:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


In [ ]:
print(test_labels)

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
773    3.0
774    0.0
775    0.0
776    0.0
777    0.0
Name: outputs, Length: 778, dtype: float64


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(awesome['labels'], awesome['outputs'], labels=[0, 1, 2, 3])

from tabulate import tabulate
data = [['Class 0', 228, 109, 139, 135],
['Class 1', 531, 4, 61, 15],
['Class 2', 323, 44, 134, 110],
['Class 3', 347, 50, 70, 144]]
print (tabulate(data, headers=["Classes", "True Positives", "True Negatives", "False Positives", "False Negatives"]))

Classes      True Positives    True Negatives    False Positives    False Negatives
---------  ----------------  ----------------  -----------------  -----------------
Class 0                 228               109                139                135
Class 1                 531                 4                 61                 15
Class 2                 323                44                134                110
Class 3                 347                50                 70                144


In [ ]:
class0 = awesome['labels']
predictions = awesome['outputs']
TruePos = np.where(class0 == predictions)[0]
FalsPositives = np.where(np.logical_and(class0 == predictions, predictions != 0))[0]
FalsNegatives = np.where(np.logical_and(class0 != predictions, predictions == 0))[0]

number_truePos = 0
for i in range (len(TruePos)):
    number_truePos += 1

number_falsPos = 0
for i in range (len(FalsPositives)):
    number_falsPos += 1

number_falsNegs = 0
for i in range (len(FalsNegatives)):
    number_falsNegs += 1

number_TrueNegs = 3112 - number_truePos - number_falsPos - number_falsNegs


TruePos = np.where(class0 == predictions)[0]
FalsPositives = np.where(np.logical_and(class0 == predictions, predictions != 1))[0]
FalsNegatives = np.where(np.logical_and(class0 != predictions, predictions == 1))[0]

number_truePos1 = 0
for i in range (len(TruePos)):
    number_truePos1 += 1

number_falsPos1 = 0
for i in range (len(FalsPositives)):
    number_falsPos1 += 1

number_falsNegs1 = 0
for i in range (len(FalsNegatives)):
    number_falsNegs1 += 1

number_TrueNegs1 = 3112 - number_truePos - number_falsPos - number_falsNegs


TruePos = np.where(class0 == predictions)[0]
FalsPositives = np.where(np.logical_and(class0 == predictions, predictions != 2))[0]
FalsNegatives = np.where(np.logical_and(class0 != predictions, predictions == 2))[0]

number_truePos2 = 0
for i in range (len(TruePos)):
    number_truePos2 += 1

number_falsPos2 = 0
for i in range (len(FalsPositives)):
    number_falsPos2 += 1

number_falsNegs2 = 0
for i in range (len(FalsNegatives)):
    number_falsNegs2 += 1

number_TrueNegs2 = 3112 - number_truePos - number_falsPos - number_falsNegs


TruePos = np.where(class0 == predictions)[0]
FalsPositives = np.where(np.logical_and(class0 == predictions, predictions != 3))[0]
FalsNegatives = np.where(np.logical_and(class0 != predictions, predictions == 3))[0]

number_truePos3 = 0
for i in range (len(TruePos)):
    number_truePos3 += 1

number_falsPos3 = 0
for i in range (len(FalsPositives)):
    number_falsPos3 += 1

number_falsNegs3 = 0
for i in range (len(FalsNegatives)):
    number_falsNegs3 += 1

number_TrueNegs3 = 3112 - number_truePos - number_falsPos - number_falsNegs


from tabulate import tabulate
data = [['Class 0', number_truePos, number_TrueNegs, number_falsPos, number_falsNegs],
['Class 1', number_truePos1, number_TrueNegs1, number_falsPos1, number_falsNegs1],
['Class 2', number_truePos2, number_TrueNegs2, number_falsPos2, number_falsNegs2],
['Class 3', number_truePos3, number_TrueNegs3, number_falsPos3, number_falsNegs3]]
print (tabulate(data, headers=["Classes", "True Positives", "True Negatives", "False Positives", "False Negatives"]))

IndexError: ignored

In [ ]:
print('Accuracy of the network on the validation set: %f %%' % (accuracy_test * 100))
print('RMSE value on validation set: %f ' % (RMSE_test))

Accuracy of the network on the validation set: 39.116203 %
RMSE value on validation set: 1.702506 


In [ ]:
class model(torch.nn.Module):
    
    def __init__(self):
        super(model, self).__init__()

        self.linear1 = torch.nn.Linear(16, 250)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(250, 70)
        self.activation = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(70, 8)
        self.activation = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(8, 1)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        x = self.activation(x)
        x = self.linear4(x)
        return x


#MODEL TRAINING


model = model()
model = model.type(torch.float64)

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)


for epoch in range(num_epochs):
    running_loss = 0.0
    for i, sample in enumerate(train_loader):
        inputs, labels = sample[:,:-1], sample[:,-1]

        inputs = inputs.type(torch.float64)
        labels = labels.type(torch.float64)

        #forward pass
        outputs = model(inputs)
        
        labels = labels.type(torch.float64)
        outputs = outputs.type(torch.float64)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()

        loss.mean().backward()
        optimizer.step()

        running_loss += loss.mean().item()
        #display loss every 100 epochs
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/100))
            running_loss = 0.0

print('Finished training')

/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([350])) that is different to the input size (torch.Size([350, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Finished training


In [ ]:
#test model on validation set
correct = 0
total = 0

#tensor to store outputs
FINAL_OUTPUTS = torch.zeros(0, dtype=torch.float32)

with torch.no_grad():
    #for sample in valid_loader:
    inputs = valid_data[:,:-1]#, valid_data[:,-1]
    FINAL_OUTPUTS = torch.cat((FINAL_OUTPUTS, model(inputs)), 0)
    #labels = labels.reshape(FINAL_OUTPUTS.size())


#set val_data = to valid_data
outputs_data = valid_data
#reshape outputs to put into val_data
outputs_data[:, -1] = torch.zeros([611,], dtype=torch.float32)
outputs_data[:, -1] = FINAL_OUTPUTS.reshape([611,])
outputs_data = np.dot(pca.transform(outputs_data)[:, :nComp], pca_components[:nComp, :])
outputs = outputs_data[:,-1]


labels = saved_labels.to_numpy()

def destandardize (Y, mean, std):
    input_destdize = (Y * std) + mean
    return input_destdize


labels = destandardize(labels, denorm_mean, denorm_std)
outputs = destandardize(outputs, denorm_mean, denorm_std)


reconstructed = pd.DataFrame()
reconstructed['outputs'] = outputs
reconstructed['labels'] = labels

for i in range(len(outputs)):
    if outputs[i] <= 25:
        outputs[i] = 0
    elif outputs[i] <= 50 and outputs[i] > 25:
        outputs[i] = 1
    elif outputs[i] <= 200 and outputs[i] > 50:
        outputs[i] = 2
    else:
        outputs[i] = 3
    if labels[i] <= 25:
        labels[i] = 0
    elif labels[i] <= 50 and labels[i] > 25:
        labels[i] = 1
    elif labels[i] <= 200 and labels[i] > 50:
        labels[i] = 2
    else:
        labels[i] = 3

finals = pd.DataFrame()
finals['outputs'] = outputs
finals['labels'] = labels
reconstructed.head()

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


,outputs,labels
0,74.285168,243.280184
1,74.545088,148.760245
2,69.098284,45.759424
3,71.722686,169.198950
4,80.294621,483.511442


In [ ]:
total = len(outputs)
correct = (outputs == labels).sum().item()

print('Accuracy of the network on the validation set: %d %%' % (
    100 * correct / total))


RMSE = np.sqrt(np.mean((labels - outputs)**2))
print('RMSE value on validation set: %f ' % (RMSE))

Accuracy of the network on the validation set: 31 %
RMSE value on validation set: 1.346633 
